# CAT-Loss War Room

> **DEMO RESEARCH MEMO** — This notebook is for demonstration purposes only.
> It is **not legal advice**. All outputs must be independently verified.
> **Verify all citations** (KeyCite / Shepardize) before any legal reliance.

---

AI-powered catastrophic loss litigation research. Given a case intake, this notebook
generates targeted research queries across weather data, carrier intel, and case law.

In [ ]:
"""Cell 1: Imports + Configuration"""
import sys, os
from pathlib import Path

# Ensure src/ is on the path
ROOT = Path(".").resolve().parent
if str(ROOT / "src") not in sys.path:
    sys.path.insert(0, str(ROOT / "src"))

from dotenv import load_dotenv
load_dotenv(ROOT / ".env")

# Config
USE_CACHE = os.getenv("USE_CACHE", "true").lower() == "true"
CACHE_DIR = str(ROOT / os.getenv("CACHE_DIR", "cache"))
CACHE_SAMPLES_DIR = str(ROOT / os.getenv("CACHE_SAMPLES_DIR", "cache_samples"))
OUTPUT_DIR = str(ROOT / os.getenv("OUTPUT_DIR", "output"))

print("=" * 50)
print("WAR ROOM CONFIG")
print("=" * 50)
print(f"  USE_CACHE:          {USE_CACHE}")
print(f"  CACHE_DIR:          {CACHE_DIR}")
print(f"  CACHE_SAMPLES_DIR:  {CACHE_SAMPLES_DIR}")
print(f"  OUTPUT_DIR:         {OUTPUT_DIR}")
print(f"  EXA_API_KEY:        {'***set***' if os.getenv('EXA_API_KEY') else 'NOT SET'}")
print("=" * 50)

In [ ]:
"""Cell 2: Case Intake"""
from war_room.query_plan import CaseIntake

intake = CaseIntake(
    event_name="Hurricane Milton",
    event_date="2024-10-09",
    state="FL",
    county="Pinellas",
    carrier="Citizens Property Insurance",
    policy_type="HO-3 Dwelling",
    posture=["denial", "bad_faith"],
    key_facts=[
        "Category 3 at landfall near Siesta Key",
        "Roof damage + water intrusion reported within 48 hours",
        "Claim denied citing pre-existing conditions",
    ],
    coverage_issues=[
        "wind vs water causation",
        "anti-concurrent causation clause",
        "duty to investigate",
    ],
)

print(intake.format_card())

In [ ]:
"""Cell 3: Query Plan Generation"""
from war_room.query_plan import generate_query_plan, format_query_plan

queries = generate_query_plan(intake)
print(format_query_plan(queries))

In [ ]:
"""Cell 4: Weather Corroboration"""
from war_room.exa_client import ExaClient
from war_room.weather_module import build_weather_brief

# Initialize Exa client (only needed if USE_CACHE=False)
client = None
try:
    client = ExaClient()
except (ValueError, Exception):
    print("No EXA_API_KEY set — running from cache only")

weather = build_weather_brief(
    intake, client,
    use_cache=USE_CACHE,
    cache_dir=CACHE_DIR,
    cache_samples_dir=CACHE_SAMPLES_DIR,
)

print(f"Event: {weather['event_summary']}")
print(f"Sources: {len(weather['sources'])}")
print(f"Metrics: {weather['metrics']}")
print()
print("Key Observations:")
for i, obs in enumerate(weather['key_observations'][:5], 1):
    print(f"  {i}. {obs[:150]}")
print()
print("Top Sources:")
for s in weather['sources'][:5]:
    print(f"  {s['badge']} {s['title'][:60]}")
    print(f"    {s['url']}")

In [ ]:
"""Cell 5: Carrier Document Pack"""
from war_room.carrier_module import build_carrier_doc_pack

carrier = build_carrier_doc_pack(
    intake, client,
    use_cache=USE_CACHE,
    cache_dir=CACHE_DIR,
    cache_samples_dir=CACHE_SAMPLES_DIR,
)

print(f"Carrier: {carrier['carrier_snapshot']['name']}")
print(f"Documents: {len(carrier['document_pack'])}")
print()
print("Document Pack:")
for d in carrier['document_pack'][:8]:
    print(f"  {d['badge']} [{d['doc_type']}] {d['title'][:60]}")
print()
print("Common Defenses:")
for d in carrier['common_defenses']:
    print(f"  - {d}")
print()
print("Rebuttal Angles:")
for r in carrier['rebuttal_angles']:
    print(f"  -> {r}")

In [ ]:
"""Cell 6: Case Law + Citation Spot-Check"""
from war_room.caselaw_module import build_caselaw_pack
from war_room.citation_verify import spot_check_citations

caselaw = build_caselaw_pack(
    intake, client,
    use_cache=USE_CACHE,
    cache_dir=CACHE_DIR,
    cache_samples_dir=CACHE_SAMPLES_DIR,
)

print("Case Law by Issue:")
for issue in caselaw['issues']:
    print(f"\n  [{issue['issue']}]")
    for c in issue['cases']:
        cite = f" -- {c['citation']}" if c.get('citation') else ''
        print(f"    {c['badge']} {c['name'][:50]}{cite}")

print("\n" + "=" * 50)
print("Citation Spot-Check")
print("=" * 50)

citecheck = spot_check_citations(
    caselaw, client,
    use_cache=USE_CACHE,
    cache_dir=CACHE_DIR,
    cache_samples_dir=CACHE_SAMPLES_DIR,
)

for c in citecheck['checks']:
    print(f"  {c['badge']} {c['case_name'][:40]} -- {c['note'][:60]}")

s = citecheck['summary']
print(f"\nSummary: {s['verified']} verified, {s['uncertain']} uncertain, {s['not_found']} not found")
print(f"\n{citecheck['disclaimer']}")

In [ ]:
"""Cell 7: Export Research Memo"""
from war_room.export_md import render_markdown_memo, write_markdown

memo_md = render_markdown_memo(
    intake, weather, carrier, caselaw, citecheck, queries
)

case_key = "milton_citizens_pinellas"
output_path = write_markdown(OUTPUT_DIR, case_key, memo_md)

print(f"Memo saved to: {output_path}")
print(f"Length: {len(memo_md)} characters")
print()
# Show first 40 lines as preview
for line in memo_md.split('\n')[:40]:
    print(line)